# DATA EDA

### Imports

In [1]:
#Imports
import pandas as pd
import numpy as np
from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer


### Dataframes Imports 

In [2]:
#importing 1st transactions
transactions_0_df= pd.read_csv("../raw_data/transactions.csv")
#Importing Transactions data
transactions_df= pd.read_csv("../raw_data/transactions_v2.csv")
#Importing user logs data
user_logs_df = pd.read_csv("../raw_data/user_logs_v2.csv")
#importing members data
members_df = pd.read_csv("../raw_data/members_v3.csv")
# import first training dataset
train_0_df= pd.read_csv("../raw_data/train.csv")
# import training dataset
train_df = pd.read_csv("../raw_data/train_v2.csv")
# import test dataset
test_df = pd.read_csv("../raw_data/sample_submission_zero.csv")
# import test dataset 2
test_2_df = pd.read_csv("../raw_data/sample_submission_v2.csv")

### Building trendline

In [5]:
#concat vertically
transactions_df_concat = pd.concat([transactions_0_df, transactions_df], axis=0)

#drop duplicates
transactions_df_global = transactions_df_concat.drop_duplicates(keep="first")

In [6]:
#concat vertically
test_df_concat = pd.concat([test_df, test_2_df], axis=0)

#drop duplicates
test_df = test_df_concat.drop_duplicates(keep="first")


In [7]:
#merge with transaction data
transactions_test_df = pd.merge(test_df, transactions_df_global, on='msno', how='left')
transactions_test_df

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,38,30,149,149,0,20170228,20170330,0
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,31,0,0,149,1,20150331,20150430,0
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,31,0,0,149,1,20150630,20150731,0
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,31,30,149,149,1,20150902,20150901,1
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,31,0,0,149,1,20150831,20150930,0
...,...,...,...,...,...,...,...,...,...,...
17130170,r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=,0,41,30,99,99,1,20170201,20170301,0
17130171,r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=,0,41,30,99,99,1,20160801,20160901,0
17130172,r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=,0,41,30,99,99,1,20160901,20161001,0
17130173,r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=,0,41,30,99,99,1,20161201,20170101,0


In [8]:
#convert to datetime
transactions_test_df.loc[:,'transaction_date'] = pd.to_datetime(transactions_test_df['transaction_date'], format='%Y%m%d')
transactions_test_df.loc[:,'membership_expire_date'] = pd.to_datetime(transactions_test_df['membership_expire_date'], format='%Y%m%d')

In [9]:
# Sort the DataFrame by 'msno' and 'transaction_date'
transactions_test_df = transactions_test_df.sort_values(by=['msno', 'transaction_date'], ascending=[True, True],)

In [10]:
#compute previous period churn
transactions_test_df['transaction_date_-1'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+1)
transactions_test_df['membership_expire_date_-1'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+1)
transactions_test_df['period_0'] = transactions_test_df['membership_expire_date_-1']-transactions_test_df['transaction_date']
transactions_test_df['period_0_churn'] = np.where(transactions_test_df['period_0'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_0'].isna(), np.nan, 0))

transactions_test_df['transaction_date_-2'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+2)
transactions_test_df['membership_expire_date_-2'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+2)
transactions_test_df['period_-1'] = transactions_test_df['membership_expire_date_-2']-transactions_test_df['transaction_date_-1']
transactions_test_df['period_-1_churn'] = np.where(transactions_test_df['period_-1'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-1'].isna(), np.nan, 0))


transactions_test_df['transaction_date_-3'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+3)
transactions_test_df['membership_expire_date_-3'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+3)
transactions_test_df['period_-2'] = transactions_test_df['membership_expire_date_-3']-transactions_test_df['transaction_date_-2']
transactions_test_df['period_-2_churn'] = np.where(transactions_test_df['period_-2'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-2'].isna(), np.nan, 0))


transactions_test_df['transaction_date_-4'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+4)
transactions_test_df['membership_expire_date_-4'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+4)
transactions_test_df['period_-3'] = transactions_test_df['membership_expire_date_-4']-transactions_test_df['transaction_date_-3']
transactions_test_df['period_-3_churn'] = np.where(transactions_test_df['period_-3'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-3'].isna(), np.nan, 0))


transactions_test_df['transaction_date_-5'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+5)
transactions_test_df['membership_expire_date_-5'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+5)
transactions_test_df['period_-4'] = transactions_test_df['membership_expire_date_-5']-transactions_test_df['transaction_date_-4']
transactions_test_df['period_-4_churn'] = np.where(transactions_test_df['period_-4'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-4'].isna(), np.nan, 0))


transactions_test_df['transaction_date_-6'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+6)
transactions_test_df['membership_expire_date_-6'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+6)
transactions_test_df['period_-5'] = transactions_test_df['membership_expire_date_-6']-transactions_test_df['transaction_date_-5']
transactions_test_df['period_-5_churn'] = np.where(transactions_test_df['period_-5'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-5'].isna(), np.nan, 0))




In [11]:
#only keeping rows with the latest transactions 
latest_transaction_indexes = transactions_test_df.groupby('msno')['transaction_date'].idxmax()
latest_transactions_with_churn = transactions_test_df.loc[latest_transaction_indexes]

In [12]:
# removing unnecessary columns
trendline_df = latest_transactions_with_churn.drop(axis=1, columns=[
    'transaction_date_-1',
    'membership_expire_date_-1',
    'period_0',
    'transaction_date_-2',
    'membership_expire_date_-2',
    'period_-1',
    'transaction_date_-3',
    'membership_expire_date_-3',
    'period_-2',
    'transaction_date_-4',
    'membership_expire_date_-4',
    'period_-3',
    'transaction_date_-5',
    'membership_expire_date_-5',
    'period_-4',
    'transaction_date_-6',
    'membership_expire_date_-6',
    'period_-5',
    'is_churn',
    'payment_method_id',
    'payment_plan_days',
    'plan_list_price',
    'actual_amount_paid',
    'is_auto_renew',
    'transaction_date',
    'membership_expire_date',
    'is_cancel'
])
                                                                        

### Transactions Data

In [13]:
###### converting transaction_date and membership_date to datetime object
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'], format='%Y%m%d')
transactions_df['membership_expire_date'] = pd.to_datetime(transactions_df['membership_expire_date'], format='%Y%m%d')

# removing duplicates and leaving only the latest transaction date
transactions_df_lt = transactions_df.loc[transactions_df.groupby('msno').transaction_date.idxmax()]

# creating remaining plan duration column
transactions_df_lt['remaining_plan_duration'] = transactions_df_lt['membership_expire_date'] - transactions_df_lt['transaction_date']

# creating discount column
transactions_df_lt['is_discount'] = transactions_df_lt.apply(lambda x: '0' if (x['actual_amount_paid'] -x['plan_list_price'])>=0 else '1', axis=1)

### User logs Data

In [14]:
# Converting date into datetime object
user_logs_df['date'] = pd.to_datetime(user_logs_df['date'], format='%Y%m%d')

#create new dataframe from transactions table with only msno and latest transaction date
latest_transactions_per_msno = transactions_df_lt[['msno', 'transaction_date']]

#Merge user logs with latest transaction date
merged_df = user_logs_df.merge(latest_transactions_per_msno, on='msno', how='left')

# drop msno's where transaction date is NaN
merged_df = merged_df.dropna(subset=['transaction_date'])

#removing rows where the user log data is before the last transaction date
user_logs_atd= merged_df.loc[merged_df['date']>=merged_df['transaction_date']]

# removing data column
user_logs_atd = user_logs_atd.drop(columns=['date','transaction_date'])

#groupby msno and summing all values
user_logs_atd = user_logs_atd.groupby('msno').sum()

### Merging

In [15]:
# Merging transactions, user logs and members data to Train dataframe
test_df = test_df.merge(transactions_df_lt, on='msno', how='left')
test_df = test_df.merge(user_logs_atd, on='msno', how='left')
test_df = test_df.merge(members_df, on='msno', how='left')
test_df = test_df.merge(trendline_df, on='msno', how='left')

### Feature engineering

In [16]:
# Drop Gender column
test_df = test_df.drop(['gender'],axis=1)

In [17]:
# Drop NaN
test_df = test_df.dropna()

In [18]:
# Changing bd outliers to NaN
test_df['bd'] = test_df['bd'].apply(lambda x: np.nan if x <14 or x > 75 else x)

In [19]:
# Converting remaining plan duration to int
test_df['remaining_plan_duration'] = test_df['remaining_plan_duration'].dt.days

In [20]:
# Converting total seconds to hours
test_df['total_secs']= round(test_df['total_secs']/3600,2)

In [21]:
# Creating average usage (hours) per day from latest transaction
test_df['usage_from_ltd'] = round(test_df['total_secs']/test_df['remaining_plan_duration'],2)
# replacing Nan with 0 
test_df['usage_from_ltd'].replace([np.nan], 0, inplace=True)
# replacing inf values with the totalsecs usage.
test_df['usage_from_ltd'] = np.where(test_df['usage_from_ltd'] == np.inf, test_df['total_secs'],test_df['usage_from_ltd'])

In [22]:
# converting registration time to datetime object
test_df['registration_init_time'] = pd.to_datetime(test_df['registration_init_time'], format='%Y%m%d')

In [23]:
# creating discount percentage column
test_df['discount_percentage'] = round((test_df['plan_list_price'] - test_df['actual_amount_paid'])/test_df['plan_list_price'],2)

test_df['discount_percentage'].replace([np.nan,-0.01,0.01], 0, inplace=True)

### Cyclic Encoding

In [24]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [25]:
# cyclic encoding transaction date
#year
test_df['last_transaction_year'] = test_df['transaction_date'].dt.year

#month
test_df['last_transaction_month'] = test_df['transaction_date'].dt.month
test_df = encode(test_df, 'last_transaction_month', 12)

#day
test_df['last_transaction_day'] = test_df['transaction_date'].dt.day
test_df= encode(test_df, 'last_transaction_day', 31)

test_df =test_df.drop(['last_transaction_month', 'last_transaction_day'],axis=1)

In [26]:
# cyclic encoding membership expire date
#year
test_df['expire_year'] = test_df['membership_expire_date'].dt.year

#month
test_df['expire_month'] = test_df['membership_expire_date'].dt.month
test_df = encode(test_df, 'expire_month', 12)

#day
test_df['expire_day'] = test_df['membership_expire_date'].dt.day
test_df= encode(test_df, 'expire_day', 31)

# droping month and day columns
test_df =test_df.drop(['expire_month', 'expire_day'],axis=1)

In [27]:
# cyclic encoding registration date
#year
test_df['registration_year'] = test_df['registration_init_time'].dt.year

#month
test_df['registration_month'] = test_df['registration_init_time'].dt.month
test_df = encode(test_df, 'registration_month', 12)

#day
test_df['registration_day'] = test_df['registration_init_time'].dt.day
test_df= encode(test_df, 'registration_day', 31)

# droping month and day columns
test_df =test_df.drop(['registration_month', 'registration_day'],axis=1)


In [28]:
# droping date columns to avoid data leakage
test_df =test_df.drop(['registration_init_time', 'membership_expire_date', 'transaction_date'],axis=1)

In [29]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 568823 entries, 2 to 1076940
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   msno                        568823 non-null  object 
 1   is_churn                    568823 non-null  int64  
 2   payment_method_id           568823 non-null  float64
 3   payment_plan_days           568823 non-null  float64
 4   plan_list_price             568823 non-null  float64
 5   actual_amount_paid          568823 non-null  float64
 6   is_auto_renew               568823 non-null  float64
 7   is_cancel                   568823 non-null  float64
 8   remaining_plan_duration     568823 non-null  int64  
 9   is_discount                 568823 non-null  object 
 10  num_25                      568823 non-null  float64
 11  num_50                      568823 non-null  float64
 12  num_75                      568823 non-null  float64
 13  num_985           

In [30]:
test_df

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,remaining_plan_duration,is_discount,...,expire_year,expire_month_sin,expire_month_cos,expire_day_sin,expire_day_cos,registration_year,registration_month_sin,registration_month_cos,registration_day_sin,registration_day_cos
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,15.0,90.0,300.0,300.0,0.0,0.0,93,0,...,2017,1.224647e-16,-1.000000e+00,0.101168,-0.994869,2013,-2.449294e-16,1.000000,-0.724793,0.688967
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0,41.0,30.0,99.0,99.0,1.0,0.0,186,0,...,2017,-1.000000e+00,-1.836970e-16,-0.485302,-0.874347,2014,5.000000e-01,0.866025,-0.937752,0.347305
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,0,40.0,30.0,149.0,149.0,1.0,1.0,-2,0,...,2017,1.000000e+00,6.123234e-17,0.299363,-0.954139,2014,5.000000e-01,0.866025,-0.848644,0.528964
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,0,38.0,90.0,477.0,477.0,0.0,0.0,101,0,...,2017,-5.000000e-01,-8.660254e-01,0.988468,0.151428,2014,8.660254e-01,0.500000,0.651372,-0.758758
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,0,40.0,30.0,149.0,149.0,1.0,0.0,60,0,...,2017,5.000000e-01,-8.660254e-01,-0.937752,0.347305,2014,8.660254e-01,0.500000,-0.571268,0.820763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076932,YYjG/92ZlsDg6ND3x+M0dQ8l0o22vhOX52TvwLjJuLg=,0,38.0,90.0,477.0,477.0,0.0,0.0,90,0,...,2017,8.660254e-01,-5.000000e-01,0.790776,-0.612106,2013,8.660254e-01,-0.500000,0.651372,-0.758758
1076933,kW9vSwigBm14tD4+SnUlgiwqQTmi0RroOX3NEpfgHUI=,0,36.0,30.0,180.0,180.0,0.0,0.0,30,0,...,2017,8.660254e-01,-5.000000e-01,-0.790776,-0.612106,2014,-2.449294e-16,1.000000,0.998717,-0.050649
1076934,d7TYCpfo982225buqtsoXCFT4zQXWCZx8rMY6xwKnPU=,0,41.0,30.0,99.0,99.0,1.0,0.0,31,0,...,2017,8.660254e-01,-5.000000e-01,0.485302,-0.874347,2015,-5.000000e-01,0.866025,0.485302,-0.874347
1076937,HTEkOM7Rl1CHqXed2XwlPeq8p/7CIM8z8d2mmRnQPMY=,0,32.0,410.0,1788.0,1788.0,0.0,0.0,413,0,...,2017,8.660254e-01,-5.000000e-01,-0.299363,-0.954139,2013,-8.660254e-01,-0.500000,-0.988468,0.151428


In [31]:
test_df.value_counts('is_churn')

is_churn
0    568823
Name: count, dtype: int64

## export not preprocessed test_df to csv

In [29]:
# # Export to CSV
# test_df.to_csv('test.csv', index=False)

## preprocess test_df

In [32]:
#dropping invaluable columns based on feature importance (see model notebooks)
test_df = test_df.drop([ 'bd', 'payment_method_id', 'city', 'registered_via'], axis=1)

In [33]:
#list features for their scalers
robust_features = [               'remaining_plan_duration', 
          'usage_from_ltd',
              'payment_plan_days',
              'plan_list_price',
              'actual_amount_paid',
             'num_50',
              'num_75',
              'num_985',
              
              'expire_year',
              'last_transaction_year'
             ]
minmax_features = ['registration_year']
normal_features = [ 'num_25',
              
              'num_100',
              'num_unq',
              'total_secs',]

In [34]:
# Build the pipeline with the different steps, this pipeline does not include the model
robust_pipeline = make_pipeline(RobustScaler())
minmax_pipeline = make_pipeline(MinMaxScaler())
log_pipeline = make_pipeline( 
    FunctionTransformer(np.log1p, validate=True),
    StandardScaler())

preprocessor_new = ColumnTransformer(
    transformers=[
        ('Robust', robust_pipeline, robust_features),
        ('MinMax', minmax_pipeline, minmax_features),
        ('Log', log_pipeline, normal_features)
    ], remainder='passthrough'
)
    

In [35]:
preprocessor_new

ColumnTransformer(remainder='passthrough',
                  transformers=[('Robust',
                                 Pipeline(steps=[('robustscaler',
                                                  RobustScaler())]),
                                 ['remaining_plan_duration', 'usage_from_ltd',
                                  'payment_plan_days', 'plan_list_price',
                                  'actual_amount_paid', 'num_50', 'num_75',
                                  'num_985', 'expire_year',
                                  'last_transaction_year']),
                                ('MinMax',
                                 Pipeline(steps=[('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['registration_year']),
                                ('Log',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer(func=<ufunc 'log1p'>,
                                                                      validate=True)),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['num_25', 'num_100', 'num_unq',
                                  'total_secs'])])

In [36]:
#drop churn column from df
test_df = test_df.drop(['is_churn'], axis=1)


In [48]:
test_df.columns

Index(['msno', 'payment_plan_days', 'plan_list_price', 'actual_amount_paid',
       'is_auto_renew', 'is_cancel', 'remaining_plan_duration', 'is_discount',
       'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs', 'period_0_churn', 'period_-1_churn', 'period_-2_churn',
       'period_-3_churn', 'period_-4_churn', 'period_-5_churn',
       'usage_from_ltd', 'discount_percentage', 'last_transaction_year',
       'last_transaction_month_sin', 'last_transaction_month_cos',
       'last_transaction_day_sin', 'last_transaction_day_cos', 'expire_year',
       'expire_month_sin', 'expire_month_cos', 'expire_day_sin',
       'expire_day_cos', 'registration_year', 'registration_month_sin',
       'registration_month_cos', 'registration_day_sin',
       'registration_day_cos'],
      dtype='object')

In [67]:
# Use fit on test_df and then transform
preprocessor_new.fit(test_df)
test_df_transformed = preprocessor_new.transform(test_df)

In [68]:
test_df_transformed[0]

array([62.0, -0.2253521126760563, 60.0, 3.02, 3.02, 1.875, 1.3,
       1.0909090909090908, 0.0, 0.0, 0.6923076923077076,
       1.0477812012338021, 0.19465802757520695, 0.49821770932211645,
       0.21206286356233442,
       'zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=', 0.0, 0.0, '0', 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 6.123233995736766e-17,
       0.29936312297335804, -0.9541392564000488, 1.2246467991473532e-16,
       -1.0, 0.10116832198743272, -0.994869323391895,
       -2.4492935982947064e-16, 1.0, -0.72479278722912,
       0.6889669190756865], dtype=object)

In [56]:
transformed_columns = robust_features + minmax_features + normal_features
transformed_columns

['remaining_plan_duration',
 'usage_from_ltd',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'num_50',
 'num_75',
 'num_985',
 'expire_year',
 'last_transaction_year',
 'registration_year',
 'num_25',
 'num_100',
 'num_unq',
 'total_secs']

In [58]:
passthrough_cols = [col for col in test_df.columns if col not in transformed_columns]
passthrough_cols

['msno',
 'is_auto_renew',
 'is_cancel',
 'is_discount',
 'period_0_churn',
 'period_-1_churn',
 'period_-2_churn',
 'period_-3_churn',
 'period_-4_churn',
 'period_-5_churn',
 'discount_percentage',
 'last_transaction_month_sin',
 'last_transaction_month_cos',
 'last_transaction_day_sin',
 'last_transaction_day_cos',
 'expire_month_sin',
 'expire_month_cos',
 'expire_day_sin',
 'expire_day_cos',
 'registration_month_sin',
 'registration_month_cos',
 'registration_day_sin',
 'registration_day_cos']

In [61]:
new_cols_order = transformed_columns + passthrough_cols
new_cols_order

['remaining_plan_duration',
 'usage_from_ltd',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'num_50',
 'num_75',
 'num_985',
 'expire_year',
 'last_transaction_year',
 'registration_year',
 'num_25',
 'num_100',
 'num_unq',
 'total_secs',
 'msno',
 'is_auto_renew',
 'is_cancel',
 'is_discount',
 'period_0_churn',
 'period_-1_churn',
 'period_-2_churn',
 'period_-3_churn',
 'period_-4_churn',
 'period_-5_churn',
 'discount_percentage',
 'last_transaction_month_sin',
 'last_transaction_month_cos',
 'last_transaction_day_sin',
 'last_transaction_day_cos',
 'expire_month_sin',
 'expire_month_cos',
 'expire_day_sin',
 'expire_day_cos',
 'registration_month_sin',
 'registration_month_cos',
 'registration_day_sin',
 'registration_day_cos']

In [69]:
# Convert transformed data back to DataFrame to ensure column order is maintained
test_df_transformed = pd.DataFrame(
    test_df_transformed, 
    columns=new_cols_order
)

In [70]:
test_df_transformed['msno']

0         zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=
1         K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=
2         ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=
3         dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=
4         otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=
                              ...                     
568818    YYjG/92ZlsDg6ND3x+M0dQ8l0o22vhOX52TvwLjJuLg=
568819    kW9vSwigBm14tD4+SnUlgiwqQTmi0RroOX3NEpfgHUI=
568820    d7TYCpfo982225buqtsoXCFT4zQXWCZx8rMY6xwKnPU=
568821    HTEkOM7Rl1CHqXed2XwlPeq8p/7CIM8z8d2mmRnQPMY=
568822    r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=
Name: msno, Length: 568823, dtype: object

## split transformed test_df into three and export CSVs

In [72]:
# Calculate split sizes
rows = len(test_df_transformed)
size = rows // 3  # Integer division

# Split the DataFrame
test_transformed_df1 = test_df_transformed.iloc[:size]
test_transformed_df2 = test_df_transformed.iloc[size:2*size]
test_transformed_df3 = test_df_transformed.iloc[2*size:]


In [73]:
# Export to CSV
test_transformed_df1.to_csv('test_transformed_df1.csv', index=False)
test_transformed_df2.to_csv('test_transformed_df2.csv', index=False)
test_transformed_df3.to_csv('test_transformed_df3.csv', index=False)